# PMI Estimation Simulation

In [1]:
import pandas as pd
from scipy.stats import weibull_min

In [2]:
def cluster(data, t_0, weights, shape_k, lambda_scale, n_comp, debug_eps = 1e-10):
    
    labels = []
    
    for i in range(len(data)):
        pre_labels = [weights[0] * \
                      weibull_min.pdf((data['D']*(data['T'] - t_0[0]))[i], \
                                      c = shape_k[0] + debug_eps, \
                                      scale = lambda_scale[0] + debug_eps)]
        for j in range(1,n_comp):
            pre_labels.append(weights[j] * \
                              weibull_min.pdf((data['D']*(data['T'] - t_0[j]))[i], \
                                              c = shape_k[j] + debug_eps, \
                                              scale = lambda_scale[j] + debug_eps))
        
        pre_labels = pre_labels / sum(pre_labels)
        pre_labels = pre_labels.tolist()
        labels.append(t_0[pre_labels.index(max(pre_labels))])
        

    return labels

def weibull_mixture_model(x, weights, shape_k, lambda_scale, n_comp):
    result = 0
    for i in range(n_comp):
        result += weights[i]*weibull_min.cdf(x, c=shape_k[i], scale=lambda_scale[i])
    return result

def quantile(q, weights, shape_k, lambda_scale, n_comp, step = 0.01, debug_eps = 1e-10):
    def condition(x):
        return weibull_mixture_model(x, weights, shape_k, lambda_scale, n_comp)
    x = step
    while condition(x) <= q:
        x += step
    return (condition(x), x)

In [3]:
necrodes = pd.read_excel("second_project/necrodes.xlsx")
creophilus = pd.read_excel("second_project/creophilus.xlsx")

necrodes = necrodes[['TEMP.', 'TIME (D)']].rename({'TEMP.': 'T', 'TIME (D)': 'D'}, axis = 1)
creophilus = creophilus[['T', 'Total immature development']].rename({'Total immature development': 'D'}, axis = 1)

In [4]:
# Creophilus
Creophilus_shape_k = [14.03, 6.72]
Creophilus_scale_lambda = [442.74, 556.66]
Creophilus_weights = [0.87, 0.13]
Creophilus_MM_t_0 = [11.25, 11.15]

dict_labs = {11.25: 'regular', 11.15: 'outliers'}

pre_Creophilus_MM_lower_k_combined = quantile(0.025, 
                               weights = Creophilus_weights, 
                               shape_k = Creophilus_shape_k,
                               lambda_scale = Creophilus_scale_lambda,
                               n_comp = 2, 
                               step = 0.01, 
                               debug_eps = 1e-10)

MM_Creophilus_lower_k_combined = pre_Creophilus_MM_lower_k_combined[1]

pre_Creophilus_MM_upper_k_combined = quantile(0.975, 
                               weights = Creophilus_weights, 
                               shape_k = Creophilus_shape_k,
                               lambda_scale = Creophilus_scale_lambda,
                               n_comp = 2, 
                               step = 0.01, 
                               debug_eps = 1e-10)

MM_Creophilus_upper_k_combined = pre_Creophilus_MM_upper_k_combined[1]

Creophilus_IT_k = 417.33
Creophilus_se = 19.52
Creophilus_IT_t_0 = 11.58
Creophilus_IT_lower_k = Creophilus_IT_k - 1.96*Creophilus_se
Creophilus_IT_upper_k = Creophilus_IT_k + 1.96*Creophilus_se

pre_labs = cluster(creophilus, 
                   Creophilus_MM_t_0, 
                   Creophilus_weights, 
                   Creophilus_shape_k, 
                   Creophilus_scale_lambda, 
                   2, 
                   debug_eps = 1e-10)

creophilus['population'] = [dict_labs[i] for i in pre_labs]
creophilus['t_0'] = pre_labs

In [5]:
creophilus['MM'] = ((creophilus['D'] < MM_Creophilus_upper_k_combined/(creophilus['T'] - creophilus['t_0'])) & (creophilus['D'] > MM_Creophilus_lower_k_combined/(creophilus['T'] - creophilus['t_0']))).astype(int)
creophilus['IT'] = ((creophilus['D'] < Creophilus_IT_upper_k/(creophilus['T'] - Creophilus_IT_t_0)) & (creophilus['D'] > Creophilus_IT_lower_k/(creophilus['T'] - Creophilus_IT_t_0))).astype(int)

In [6]:
creophilus_regular = creophilus[creophilus['population'] == 'regular'].copy()
creophilus_outliers = creophilus[creophilus['population'] == 'outliers'].copy()

In [7]:
k_upper = weibull_min.ppf(0.975, c=Creophilus_shape_k[0], scale=Creophilus_scale_lambda[0])
k_lower = weibull_min.ppf(0.025, c=Creophilus_shape_k[0], scale=Creophilus_scale_lambda[0])

creophilus_regular['MM_regular'] = ((creophilus_regular['D'] < k_upper/(creophilus_regular['T'] - creophilus_regular['t_0'])) & (creophilus_regular['D'] > k_lower/(creophilus_regular['T'] - creophilus_regular['t_0']))).astype(int)

k_upper = weibull_min.ppf(0.975, c=Creophilus_shape_k[1], scale=Creophilus_scale_lambda[1])
k_lower = weibull_min.ppf(0.025, c=Creophilus_shape_k[1], scale=Creophilus_scale_lambda[1])

creophilus_outliers['MM_outliers'] = ((creophilus_outliers['D'] < k_upper/(creophilus_outliers['T'] - creophilus_outliers['t_0'])) & (creophilus_outliers['D'] > k_lower/(creophilus_outliers['T'] - creophilus_outliers['t_0']))).astype(int)

In [28]:
summary_creophilus = (
    creophilus.groupby("T", as_index=False)
      .agg({"MM": "mean", "IT": "mean", "T": "mean"})
)
summary_creophilus

,MM,IT,T
0,0.857143,0.428571,15.0
1,0.809524,0.333333,17.5
2,1.000000,0.375000,20.0
3,1.000000,0.700000,22.5
4,1.000000,0.833333,25.0
5,1.000000,0.555556,27.5
6,1.000000,0.846154,30.0


In [9]:
creophilus[['MM', 'IT']].mean(axis=0)

MM    0.965318
IT    0.589595
dtype: float64

In [29]:
summary_creophilus_regular = (
    creophilus_regular.groupby("T", as_index=False)
      .agg({"MM_regular": "mean", "T": "mean"})
)
summary_creophilus_regular

,MM_regular,T
0,0.777778,15.0
1,0.705882,17.5
2,1.000000,20.0
3,1.000000,22.5
4,1.000000,25.0
5,0.909091,27.5
6,0.923077,30.0


In [24]:
creophilus_regular[['MM_regular']].mean(axis=0)

MM_regular    0.937107
dtype: float64

In [30]:
summary_creophilus_outliers = (
    creophilus_outliers.groupby("T", as_index=False)
      .agg({"MM_outliers": "mean", "T": "mean"})
)
summary_creophilus_outliers

,MM_outliers,T
0,0.80,15.0
1,0.75,17.5
2,1.00,27.5


In [25]:
creophilus_outliers[['MM_outliers']].mean(axis=0)

MM_outliers    0.857143
dtype: float64

In [14]:
# Necrodes
Necrodes_shape_k = [14.57, 13.67]
Necrodes_scale_lambda = [394.44, 423.95]
Necrodes_weights = [0.41, 0.59]
Necrodes_MM_t_0 = [9.52, 10.21]

dict_labs = {9.52: 'large', 10.21: 'small'}

pre_Necrodes_MM_lower_k_combined = quantile(0.025, 
                               weights = Necrodes_weights, 
                               shape_k = Necrodes_shape_k,
                               lambda_scale = Necrodes_scale_lambda,
                               n_comp = 2, 
                               step = 0.01, 
                               debug_eps = 1e-10)

MM_Necrodes_lower_k_combined = pre_Necrodes_MM_lower_k_combined[1]

pre_Necrodes_MM_upper_k_combined = quantile(0.975, 
                               weights = Necrodes_weights, 
                               shape_k = Necrodes_shape_k,
                               lambda_scale = Necrodes_scale_lambda,
                               n_comp = 2, 
                               step = 0.01, 
                               debug_eps = 1e-10)

MM_Necrodes_upper_k_combined = pre_Necrodes_MM_upper_k_combined[1]

Necrodes_IT_k = 468.89
Necrodes_se = 24.59
Necrodes_IT_t_0 = 8.49
Necrodes_IT_lower_k = Necrodes_IT_k - 1.96*Necrodes_se
Necrodes_IT_upper_k = Necrodes_IT_k + 1.96*Necrodes_se

pre_labs = cluster(necrodes, 
                   Necrodes_MM_t_0, 
                   Necrodes_weights, 
                   Necrodes_shape_k, 
                   Necrodes_scale_lambda, 
                   2, 
                   debug_eps = 1e-10)
necrodes['population'] = [dict_labs[i] for i in pre_labs]
necrodes['t_0'] = pre_labs

In [15]:
necrodes['MM'] = ((necrodes['D'] < MM_Necrodes_upper_k_combined/(necrodes['T'] - necrodes['t_0'])) & (necrodes['D'] > MM_Necrodes_lower_k_combined/(necrodes['T'] - necrodes['t_0']))).astype(int)
necrodes['IT'] = ((necrodes['D'] < Necrodes_IT_upper_k/(necrodes['T'] - Necrodes_IT_t_0)) & (necrodes['D'] > Necrodes_IT_lower_k/(necrodes['T'] - Necrodes_IT_t_0))).astype(int)

In [16]:
necrodes_large = necrodes[necrodes['population'] == 'large'].copy()
necrodes_small = necrodes[necrodes['population'] == 'small'].copy()

In [17]:
k_upper = weibull_min.ppf(0.975, c=Necrodes_shape_k[0], scale=Necrodes_scale_lambda[0])
k_lower = weibull_min.ppf(0.025, c=Necrodes_shape_k[0], scale=Necrodes_scale_lambda[0])

necrodes_large['MM_large'] = ((necrodes_large['D'] < k_upper/(necrodes_large['T'] - necrodes_large['t_0'])) & (necrodes_large['D'] > k_lower/(necrodes_large['T'] - necrodes_large['t_0']))).astype(int)

k_upper = weibull_min.ppf(0.975, c=Necrodes_shape_k[1], scale=Necrodes_scale_lambda[1])
k_lower = weibull_min.ppf(0.025, c=Necrodes_shape_k[1], scale=Necrodes_scale_lambda[1])

necrodes_small['MM_small'] = ((necrodes_small['D'] < k_upper/(necrodes_small['T'] - necrodes_small['t_0'])) & (necrodes_small['D'] > k_lower/(necrodes_small['T'] - necrodes_small['t_0']))).astype(int)

In [31]:
summary_necrodes = (
    necrodes.groupby("T", as_index=False)
      .agg({"MM": "mean", "IT": "mean", "T": "mean"})
)
summary_necrodes

,MM,IT,T
0,1.00,0.87037,14.0
1,1.00,0.96000,15.0
2,1.00,0.77000,16.0
3,1.00,0.26000,17.0
4,0.99,0.91000,18.0
5,1.00,0.99000,19.0
6,1.00,0.79000,20.0
7,1.00,0.02000,22.0
8,0.97,0.97000,26.0
9,0.79,1.00000,30.0


In [19]:
necrodes[['MM', 'IT']].mean(axis=0)

MM    0.973795
IT    0.748428
dtype: float64

In [32]:
summary_necrodes_large = (
    necrodes_large.groupby("T", as_index=False)
      .agg({"MM_large": "mean", "T": "mean"})
)
summary_necrodes_large

,MM_large,T
0,1.0,14.0
1,1.0,15.0
2,1.0,16.0
3,1.0,18.0
4,1.0,19.0
5,1.0,22.0
6,1.0,26.0


In [26]:
necrodes_large[['MM_large']].mean(axis=0)

MM_large    1.0
dtype: float64

In [33]:
summary_necrodes_small = (
    necrodes_small.groupby("T", as_index=False)
      .agg({"MM_small": "mean", "T": "mean"})
)
summary_necrodes_small

,MM_small,T
0,1.000000,14.0
1,1.000000,17.0
2,1.000000,18.0
3,1.000000,19.0
4,1.000000,20.0
5,0.956522,26.0
6,0.820000,30.0


In [23]:
necrodes_small[['MM_small']].mean(axis=0)

MM_small    0.957055
dtype: float64